# Load Data 

In [ ]:
!pip install pyclustering

import pandas as pd 
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

import numpy as np
import math

import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 42.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# File path set and defined to Google Drive to access all the datasets.
file_path = '/content/drive/MyDrive/IST340_Spring_Lab/'

In [8]:
df = pd.read_csv(file_path + 'df_b_pos.csv')
df_input=df.copy()
df_input.head()

,no_atbat,no_hits,no_home,no_runs,no_rbi,no_bb,yr_major,cr_atbat,cr_hits,cr_home,cr_runs,cr_rbi,cr_bb,no_outs,no_assts,no_error,logsalar,league_American,league_National,division_East,division_West,team_Atlanta,team_Baltimore,team_Boston,team_California,team_Chicago,team_Cincinnati,team_Cleveland,team_Detroit,team_Houston,team_KansasCity,team_LosAngeles,team_Milwaukee,team_Minneapolis,team_Montreal,team_NewYork,team_Oakland,team_Philadelphia,team_Pittsburgh,team_SanDiego,team_SanFrancisco,team_Seattle,team_StLouis,team_Texas,team_Toronto,position_1B,position_2B,position_3B,position_C,position_CF,position_DH,position_LF,position_OF,position_RF,position_SS,position_UT
0,293,66,1,30,29,14,1,293,66,1,30,29,14,446,33,20,NaN,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,6.163315,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,6.173786,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,6.214608,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,4.516339,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


# Missing Value (Missing Forrest)

In [9]:
!pip install scikit-learn==1.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [10]:
!pip install missingpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.4 MB/s eta 0:00:00


In [11]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

from missingpy import MissForest

In [12]:
# Make an instance and perform the imputation
# Use all default values for the imputer
tree_imputer = MissForest()

X = df_input.drop(['no_atbat','no_hits','no_home','no_runs','no_rbi','yr_major','cr_atbat','cr_hits','cr_home','cr_runs','cr_rbi',
                   'cr_bb','no_outs','no_assts','no_error','league_American','league_National','division_East','division_West',
                   'team_Atlanta','team_Baltimore','team_Boston','team_California','team_Chicago','team_Cincinnati','team_Cleveland',
                   'team_Detroit','team_Houston','team_KansasCity','team_LosAngeles','team_Milwaukee','team_Minneapolis','team_Montreal',
                   'team_NewYork','team_Oakland','team_Philadelphia','team_Pittsburgh','team_SanDiego','team_SanFrancisco','team_Seattle',
                   'team_StLouis','team_Texas','team_Toronto','position_1B','position_2B','position_3B','position_C','position_CF',
                   'position_DH','position_LF','position_OF','position_RF','position_SS','position_UT'], axis=1,) #drop the columns with no missing value

In [13]:
tree_impute_array = tree_imputer.fit_transform(X)

Iteration: 0
Iteration: 1
Iteration: 2


In [14]:
df_input['logsalar'] = tree_impute_array [:, 1]
#I chose 1 because Iteration 0 gave values that were way too high compared to the logsalar values in the original dataset

In [15]:
# Create a metadata dataframe that contains information about the input dataframe
def metadata(df):
    columns_list = list(df.columns.values)
    metadata = pd.DataFrame(columns_list, columns=['col_name'])

    # Data types
    metadata['data_type'] = [i for i in df.dtypes.astype(str)]

    # Missing values
    metadata['missing_values'] = [i for i in df.isnull().sum()]

    # Missing values percentage in two digits number
    metadata['missing_values_percentage'] = [round(i*100,2) for i in df.isnull().sum() / df.shape[0]]
    
    # Number of unique values
    metadata['unique_values'] = [i for i in df.nunique()]

    # Number of unique values percentage
    metadata['unique_values_percentage'] = [round(i*100,2) for i in df.nunique() / df.shape[0]]

    # Basic statistics for interval columns
    interval_columns = [col for col in columns_list if df[col].dtype != 'object']
    desc_interval = df[interval_columns].agg(['min', 'max', 'std']).transpose().reset_index().rename(columns={'index': 'col_name'})
    metadata = metadata.merge(desc_interval, on='col_name', how='left')

    return metadata

In [16]:
#Dataframe with columns that do not have missing values
Tree_impute_df = df_input[['logsalar','no_atbat','no_hits','no_home','no_runs','no_rbi','yr_major','cr_atbat','cr_hits','cr_home','cr_runs','cr_rbi',
                   'cr_bb','no_outs','no_assts','no_error','league_American','league_National','division_East','division_West',
                   'team_Atlanta','team_Baltimore','team_Boston','team_California','team_Chicago','team_Cincinnati','team_Cleveland',
                   'team_Detroit','team_Houston','team_KansasCity','team_LosAngeles','team_Milwaukee','team_Minneapolis','team_Montreal',
                   'team_NewYork','team_Oakland','team_Philadelphia','team_Pittsburgh','team_SanDiego','team_SanFrancisco','team_Seattle',
                   'team_StLouis','team_Texas','team_Toronto','position_1B','position_2B','position_3B','position_C','position_CF',
                   'position_DH','position_LF','position_OF','position_RF','position_SS','position_UT']]
Tree_impute_df.head(7)

,logsalar,no_atbat,no_hits,no_home,no_runs,no_rbi,yr_major,cr_atbat,cr_hits,cr_home,cr_runs,cr_rbi,cr_bb,no_outs,no_assts,no_error,league_American,league_National,division_East,division_West,team_Atlanta,team_Baltimore,team_Boston,team_California,team_Chicago,team_Cincinnati,team_Cleveland,team_Detroit,team_Houston,team_KansasCity,team_LosAngeles,team_Milwaukee,team_Minneapolis,team_Montreal,team_NewYork,team_Oakland,team_Philadelphia,team_Pittsburgh,team_SanDiego,team_SanFrancisco,team_Seattle,team_StLouis,team_Texas,team_Toronto,position_1B,position_2B,position_3B,position_C,position_CF,position_DH,position_LF,position_OF,position_RF,position_SS,position_UT
0,5.138534,293,66,1,30,29,1,293,66,1,30,29,14,446,33,20,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6.163315,315,81,7,24,38,14,3449,835,69,321,414,375,632,43,10,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,6.173786,479,130,18,66,72,3,1624,457,63,224,266,263,880,82,14,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,6.214608,496,141,20,65,78,11,5628,1575,225,828,838,354,200,11,3,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,4.516339,321,87,10,39,42,2,396,101,12,48,46,33,805,40,4,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,6.620073,594,169,4,74,51,11,4408,1133,19,501,336,194,282,421,25,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,4.248495,185,37,1,23,8,2,214,42,1,30,9,24,76,127,7,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [17]:
metadata(Tree_impute_df)

,col_name,data_type,missing_values,missing_values_percentage,unique_values,unique_values_percentage,min,max,std
0,logsalar,float64,0,0.0,187,58.07,4.212128,7.807917,0.838737
1,no_atbat,int64,0,0.0,246,76.40,127.000000,687.000000,143.595835
2,no_hits,int64,0,0.0,136,42.24,31.000000,238.000000,44.179509
3,no_home,int64,0,0.0,37,11.49,0.000000,40.000000,8.698770
4,no_runs,int64,0,0.0,91,28.26,12.000000,130.000000,25.057366
5,no_rbi,int64,0,0.0,100,31.06,8.000000,121.000000,25.501162
6,yr_major,int64,0,0.0,22,6.83,1.000000,24.000000,4.969707
7,cr_atbat,int64,0,0.0,315,97.83,166.000000,14053.000000,2328.479167
8,cr_hits,int64,0,0.0,287,89.13,34.000000,4256.000000,654.787619
9,cr_home,int64,0,0.0,151,46.89,0.000000,548.000000,90.065127


In [18]:
# Write file to CSV
Tree_impute_df.to_csv(file_path + 'TaskB.csv', index=False)